In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Specifiy dementia subtype

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

## Specify exposure window

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

## Load relevant dataframe and create intercept

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_' + subtype + '_' + window +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [8]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,other_sedatives_100_pdds,antipsychotics_100_pdds,depot_antipsychotics_100_pdds,antidepressants_100_pdds
17237,3220538,31,538,1,2008-10-15,False,NaN,1998-08-21,2012-12-13,4697,77,0.0,0.0,0.0,0.000000,0.0,0.0
12235,15541330,43,330,0,2011-03-24,True,4693.0,2000-03-22,2013-05-09,50213,68,0.0,0.0,0.0,0.227273,0.0,0.0
6780,5945377,35,377,0,2011-05-05,False,NaN,2001-04-01,2013-05-08,41103,76,0.0,0.0,0.0,0.000000,0.0,0.0
7994,1838066,18,66,1,2010-11-18,False,NaN,1999-12-13,2012-04-16,13064,92,0.0,0.0,0.0,0.000000,0.0,0.0
12762,6828315,27,315,1,2009-12-29,False,NaN,1999-11-26,2013-02-14,33131,82,0.0,0.0,0.0,0.000000,0.0,0.0


In [9]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'other_sedatives_100_pdds',
       'antipsychotics_100_pdds', 'depot_antipsychotics_100_pdds',
       'antidepressants_100_pdds'],
      dtype='object')

In [10]:
pt_features['intercept'] = 1.0

In [11]:
# pt_features.sort_values(by='insomnia_count<=10',ascending=False)

In [12]:
columns_always_for_inclusion = [
    'female',
    'non_insomnia_GP_consultations', 'stroke',
    'non_stroke_vascular_disease', 'hypertension',
    'diabetes', 'mental_illness_non_smi', 'mental_illness_smi',
    'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
    'mood_stabilisers_100_pdds',
    'other_sedatives_100_pdds', 'antidepressants_100_pdds', 
    'depot_antipsychotics_100_pdds', 'antipsychotics_100_pdds',
    'benzo_and_z_drugs_100_pdds'
]

## Insomnia consultations

### insomnia yes/no

In [13]:
cols_for_inclusion_here = ['insomnia_any','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean < 0
mood_stabilisers_100_pdds  being retained as mean < 0
benzo_and_z_drugs_100_pdds  being retained as mean < 0
other_sedatives_100_pdds  being retained as mean < 0
antipsychotics_100_pdds  being retained as mean < 0
depot_antipsychotics_100_pdds  being removed as mean = 0
antidepressants_100_pdds  being retained as mean < 0


*Stage 1*
Univariate results
                            Univariate OR  p value  [0.025    0.975]
antidepressants_100_pdds            1.042    0.000   1.029     1.056
antipsychotics_100_pdds             1.001    0.898   0.980     1.024
benzo_and_z_drugs_100_pdds          1.045    0.176   0.980     1.114
female                              1.000    1.000   0.968     1.033
mood_stabilisers_100_pdds           7.239    0.485   0.028  1869.422
other_sedatives_100_pdds            1.041    0.538   0.916     1.183

The following covariates were retained during stage 1:

['antidepressants_100_pdds', 'benzo_and_z_drugs_100_pdds']

The follow

In [14]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
antidepressants_100_pdds,1.042,0.000,1.029,1.056,1.042,0.000,1.029,1.056
benzo_and_z_drugs_100_pdds,1.045,0.176,0.980,1.114,1.044,0.184,0.980,1.112


### insomnia (continuous variable)

In [15]:
cols_for_inclusion_here = ['insomnia','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean < 0
mood_stabilisers_100_pdds  being retained as mean < 0
benzo_and_z_drugs_100_pdds  being retained as mean < 0
other_sedatives_100_pdds  being retained as mean < 0
antipsychotics_100_pdds  being retained as mean < 0
depot_antipsychotics_100_pdds  being removed as mean = 0
antidepressants_100_pdds  being retained as mean < 0


*Stage 1*
Univariate results
                            Univariate OR  p value  [0.025    0.975]
antidepressants_100_pdds            1.042    0.000   1.029     1.056
antipsychotics_100_pdds             1.001    0.898   0.980     1.024
benzo_and_z_drugs_100_pdds          1.045    0.176   0.980     1.114
female                              1.000    1.000   0.968     1.033
mood_stabilisers_100_pdds           7.239    0.485   0.028  1869.422
other_sedatives_100_pdds            1.041    0.538   0.916     1.183

The following covariates were retained during stage 1:

['antidepressants_100_pdds', 'benzo_and_z_drugs_100_pdds']

The follow

In [16]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
antidepressants_100_pdds,1.042,0.000,1.029,1.056,1.042,0.000,1.029,1.056
benzo_and_z_drugs_100_pdds,1.045,0.176,0.980,1.114,1.044,0.184,0.980,1.112


### insomnia quantiles

In [17]:
cols_for_inclusion_here = ['benzo_and_z_drugs_100_pdds','insomnia_count:1_5','insomnia_count:6_10','insomnia_count:above_10']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean < 0
mood_stabilisers_100_pdds  being retained as mean < 0
benzo_and_z_drugs_100_pdds  being retained as mean < 0
other_sedatives_100_pdds  being retained as mean < 0
antipsychotics_100_pdds  being retained as mean < 0
depot_antipsychotics_100_pdds  being removed as mean = 0
antidepressants_100_pdds  being retained as mean < 0


*Stage 1*
Univariate results
                            Univariate OR  p value  [0.025    0.975]
antidepressants_100_pdds            1.042    0.000   1.029     1.056
antipsychotics_100_pdds             1.001    0.898   0.980     1.024
benzo_and_z_drugs_100_pdds          1.045    0.176   0.980     1.114
female                              1.000    1.000   0.968     1.033
mood_stabilisers_100_pdds           7.239    0.485   0.028  1869.422
other_sedatives_100_pdds            1.041    0.538   0.916     1.183

The following covariates were retained during stage 1:

['antidepressants_100_pdds', 'benzo_and_z_drugs_100_pdds']

The follow

In [18]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
antidepressants_100_pdds,1.042,0.000,1.029,1.056,1.042,0.000,1.029,1.056
benzo_and_z_drugs_100_pdds,1.045,0.176,0.980,1.114,1.044,0.184,0.980,1.112


## Benzo and z-drug PDDs

In [19]:
### benzos yes/no

In [20]:
# cols_for_inclusion_here = ['benzo_and_z_drugs_any']
# training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
# summary_table,result = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [21]:
# result

### benzo PDDs

In [22]:
# cols_for_inclusion_here = ['benzo_and_z_drugs_pdds']
# training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
# summary_table,result = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [23]:
# result

### benzos quantiles

In [24]:
# cols_for_inclusion_here = ['benzo_and_z_drugs_pdds:1_10', 'benzo_and_z_drugs_pdds:11_100',
#        'benzo_and_z_drugs_pdds:101_1000', 'benzo_and_z_drugs_pdds:1001_10000',
#        'benzo_and_z_drugs_pdds:above_10000']
# training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
# summary_table,result = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [25]:
# result

'antidepressants'